<a href="https://colab.research.google.com/github/Saponjyan/CV/blob/main/alpaca_lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Talk to Alpaca-LoRA

This notebook contains minimal code for running [Alpaca-LoRA](https://github.com/tloen/alpaca-lora/) for demonstration purposes. Please check the repo for more details.

In [6]:
# !pip install bitsandbytes
# !pip install -q datasets loralib sentencepiece
# !pip install -q git+https://github.com/zphang/transformers@c3dc391
# !pip install -q git+https://github.com/huggingface/peft.git


In [8]:
from peft import PeftModel
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig

tokenizer = LLaMATokenizer.from_pretrained("decapoda-research/llama-7b-hf")
model = LLaMAForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)
model = PeftModel.from_pretrained(model, "tloen/alpaca-lora-7b")

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 5>:5                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2578 in from_pretrained   │
│                                                                                                  │
│   2575 │   │   │   │   │   key: device_map[key] for key in device_map.keys() if key not in modu  │
│   2576 │   │   │   │   }                                                                         │
│   2577 │   │   │   │   if "cpu" in device_map_without_lm_head.values() or "disk" in device_map_  │
│ ❱ 2578 │   │   │   │   │   raise ValueError(                                                     │
│   2579 │   │   │   │   │   │   """                                                               │
│   2580 │   │   │   │   │   │   Some modules are dispatched on the CPU or the disk. Make sure yo  │
│   2581 │   │   │   │   │   │   the quantized model. If you have set a value for `max_memory` yo  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: 
                        Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to 
fit
                        the quantized model. If you have set a value for `max_memory` you should increase that. To 
have
                        an idea of the modules that are set on the CPU or RAM you can print model.hf_device_map.

In [ ]:
def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:"""

In [9]:
generation_config = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    num_beams=4,
)

def evaluate(instruction, input=None):
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256
    )
    for s in generation_output.sequences:
        output = tokenizer.decode(s)
        print("Response:", output.split("### Response:")[1].strip())

In [ ]:
f =True
while f:
  evaluate(input("Instruction: "))

Instruction: how are you&
Response: I'm doing well, thanks for asking!
Instruction: как дела?
Response: How are you?
Instruction: чем занят?
Response: What are you doing?
Instruction: црфе фку нщг вщштп?
Response: What is the meaning of the phrase "црфе фку нщг вщштп"?
Instruction: what ar5e you doing&
Response: I'm working on a project.
Instruction: what project?
Response: I am currently working on a project to develop a mobile app.
Instruction: who are you?
Response: I am a curious and creative individual who loves to learn and explore.
Instruction: what is yor name&
Response: John
Instruction: what is your surename?
Response: Smith
Instruction: can you drow image?
Response: Yes, I can.
Instruction: drow pleas
Response: Please do not drop the ball.
Instruction: can i change your code?
Response: Yes, you can change my code.
Instruction: how?
Response: What is the best way to solve this problem?
Instruction: what langueges you know&
Response: I know English, Spanish, French, and German